# Section 1: File Interity checker
This notebook scans the `files_to_hash` folder, computes SHA-256 digests, and records the results with timestamps in `hash_results.csv`.

In [5]:
from __future__ import annotations

import csv
import hashlib
from datetime import datetime
from pathlib import Path

SOURCE_DIR = Path.cwd() / "files_to_hash"
OUTPUT_CSV = Path.cwd() / "hash_results.csv"

def sha256_file(path: Path) -> str:
    """Return the SHA-256 hex digest of the file at `path`."""
    hasher = hashlib.sha256()
    with path.open("rb") as handle:
        for chunk in iter(lambda: handle.read(8192), b""):
            hasher.update(chunk)
    return hasher.hexdigest()

In [6]:
if not SOURCE_DIR.exists():
    raise FileNotFoundError(f"Expected directory missing: {SOURCE_DIR}")

files = sorted([p for p in SOURCE_DIR.iterdir() if p.is_file()], key=lambda p: p.name.lower())
timestamp = datetime.utcnow().isoformat(timespec="seconds") + "Z"

OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)

records = []
for file_path in files:
    digest = sha256_file(file_path)
    records.append({
        "filename": file_path.name,
        "sha256": digest,
        "timestamp": timestamp,
    })

with OUTPUT_CSV.open("w", newline="", encoding="utf-8") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["filename", "sha256", "timestamp"])
    writer.writeheader()
    writer.writerows(records)

print(f"Processed {len(records)} files. Results saved to {OUTPUT_CSV}")
records

Processed 3 files. Results saved to c:\Users\Micha\OneDrive\My Time at Goldsmiths\Year 3\Networks and System Security\E-Portfolio of Evidence\Week 04\hash_results.csv


C:\Users\Micha\AppData\Local\Temp\ipykernel_22284\315605800.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat(timespec="seconds") + "Z"


[{'filename': 'checklist.md',
  'sha256': '854a628d5c411b0000838771adbff3a7657ddf50a7ac22c6b18e6ed535cd413c',
  'timestamp': '2025-12-10T12:58:22Z'},
 {'filename': 'network_notes.txt',
  'sha256': '801ee3a1b47e77ddf9d0db5ebd932ab6fac783b69feadf20b0a31085a8331519',
  'timestamp': '2025-12-10T12:58:22Z'},
 {'filename': 'password_policies.txt',
  'sha256': 'bd8970a3858ab0d1659505cf4ec59fabae1db068f0c06b58d701fde8c09aa687',
  'timestamp': '2025-12-10T12:58:22Z'}]

## Section 2: Detecting Suspicious File Changes 

In [7]:
baseline_records = {record["filename"]: record for record in records}

# Simulate modified and missing files relative to baseline
current_snapshot = {
    entry["filename"]: entry
    for entry in [
        {"filename": "network_notes.txt", "sha256": sha256_file(SOURCE_DIR / "network_notes.txt"), "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z"},
        {"filename": "password_policies.txt", "sha256": "DEADBEEF", "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z"},
        # checklist.md missing on purpose to simulate deletion
    ]
}

changes = {"modified": [], "missing": [], "new": []}

for filename, baseline_entry in baseline_records.items():
    current_entry = current_snapshot.get(filename)
    if not current_entry:
        changes["missing"].append(filename)
        continue
    if current_entry["sha256"].lower() != baseline_entry["sha256"].lower():
        changes["modified"].append(filename)

for filename in current_snapshot.keys() - baseline_records.keys():
    changes["new"].append(filename)

changes

C:\Users\Micha\AppData\Local\Temp\ipykernel_22284\627098694.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  {"filename": "network_notes.txt", "sha256": sha256_file(SOURCE_DIR / "network_notes.txt"), "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z"},
C:\Users\Micha\AppData\Local\Temp\ipykernel_22284\627098694.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  {"filename": "password_policies.txt", "sha256": "DEADBEEF", "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z"},


{'modified': ['password_policies.txt'], 'missing': ['checklist.md'], 'new': []}

## Section 3: Signature-Based Malware Detection

In [8]:
import re

SIGNATURES = [r"eval\(", r"base64\.b64decode", r"socket\.connect", r"exec\(", r"import os"]
signature_patterns = [re.compile(sig) for sig in SIGNATURES]

def scan_file_for_signatures(path: Path) -> Dict[str, int]:
    """Return a dict of signature matches counts for the given file."""
    try:
        text = path.read_text(encoding="utf-8", errors="ignore")
    except Exception as exc:
        print(f"Could not read {path}: {exc}")
        return {}

    hits: Dict[str, int] = {}
    for pattern, signature in zip(signature_patterns, SIGNATURES):
        matches = pattern.findall(text)
        if matches:
            hits[signature] = len(matches)
    return hits

scan_results = {}
for file_path in SOURCE_DIR.iterdir():
    if not file_path.is_file():
        continue
    matches = scan_file_for_signatures(file_path)
    if matches:
        scan_results[file_path.name] = matches

scan_results if scan_results else "No suspicious patterns found."

'No suspicious patterns found.'

## Section 4: Worm Propagation Simulation

In [ ]:
import random
from collections import Counter

def simulate_worm(total_hosts: int = 100, initially_infected: int = 1, steps: int = 20, scan_attempts_per_step: int = 5) -> Counter:
    """Simulate worm spread by random scanning across hosts."""
    infected = set(random.sample(range(total_hosts), initially_infected))
    history = Counter({"infected": len(infected)})

    for step in range(1, steps + 1):
        new_infections = set()
        for host in infected:
            for _ in range(scan_attempts_per_step):
                target = random.randrange(total_hosts)
                if target not in infected:
                    new_infections.add(target)
        infected |= new_infections
        history[step] = len(infected)
        if len(infected) == total_hosts:
            break

    return history

random.seed(42)
propagation_history = simulate_worm(total_hosts=200, initially_infected=3, steps=15, scan_attempts_per_step=10)
propagation_history